In [ ]:
import streamlit as st
import cv2
import numpy as np
import tempfile
from tensorflow.keras.models import load_model

# Load trained model once
@st.cache_resource
def load_violence_model():
    model = load_model("violence_detection_model (1).h5")
    return model

model = load_violence_model()

# Frame extraction function
def extract_frames_from_video(video_path, num_frames=16, target_size=(64, 64)):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames < num_frames:
        return None

    frame_indices = np.linspace(0, total_frames - 1, num_frames).astype(int)
    frames = []
    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_indices:
            frame = cv2.resize(frame, target_size)
            frame = frame / 255.0
            frames.append(frame)

    cap.release()
    return np.array(frames)

# Prediction function
def predict_violence(frames, model):
    if frames.shape != (16, 64, 64, 3):
        return " Frame input shape mismatch"
    input_data = np.expand_dims(frames, axis=0)
    prediction = model.predict(input_data)[0][0]
    return "⚠️ Fight Detected!" if prediction > 0.5 else "✅ No Fight Detected."

# Streamlit UI
st.set_page_config(page_title="Violence Detection", layout="centered")
st.title("🔍 Violence Detection in Videos")

video_file = st.file_uploader("Upload a Video", type=["mp4", "avi", "mov"])

if video_file:
    with tempfile.NamedTemporaryFile(delete=False, suffix='.mp4') as temp_video:
        temp_video.write(video_file.read())
        video_path = temp_video.name

    st.video(video_file)
    st.write(" Extracting frames and running prediction...")

    frames = extract_frames_from_video(video_path)

    if frames is None or len(frames) < 16:
        st.error("❌ Couldn't extract enough frames from the video.")
    else:
        result = predict_violence(frames, model)
        st.success(f" Prediction Result: {result}")